In [4]:
# %matplotlib widget for Voilà compatibility
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output
from io import BytesIO
from PIL import Image

# Global widgets
file_uploader = widgets.FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
load_image_button = widgets.Button(description="Load Image", button_style="primary")
report_output = widgets.Output()
status_output = widgets.Output()
plot_output = widgets.Output()

# Global variables
fig, ax = None, None
img = None
clicked_points = []
affine_matrices = []
red_annotations = []
green_annotations = []
calibration_df = pd.DataFrame(columns=["label", "original_x", "original_y"])
measurement_df = pd.DataFrame(columns=["label", "original_x", "original_y"])

# Additional widgets
click_mode_toggle = widgets.ToggleButton(value=False, description="Add Point (click)")
man_x_input = widgets.FloatText(description='X', step=0.1)
man_y_input = widgets.FloatText(description='Y', step=0.1)
alt_sys_dropdown = widgets.Dropdown(options=["None"], description='System')
man_add_btn = widgets.Button(description="Add Point")
save_btn = widgets.Button(description="Save Image + Report", button_style='info')

coord_input_fields = [widgets.HBox([widgets.FloatText(description=f'X{i+1}'),
                                    widgets.FloatText(description=f'Y{i+1}')]) for i in range(3)]
sys_input = widgets.Text(description='System', placeholder='e.g., µ-Raman')
affine_compute_btn = widgets.Button(description="Compute Transform")
clear_red_btn = widgets.Button(description="Clear Calib", button_style='danger')
clear_green_btn = widgets.Button(description="Clear Meas", button_style='success')

manual_input_layout = widgets.VBox([
    widgets.Label("Add a point using manual coordinates (image or transformed):"),
    widgets.HBox([man_add_btn, man_x_input, man_y_input, alt_sys_dropdown])
])
click_input_layout = widgets.VBox([
    widgets.Label("Add a point by clicking on the image:"),
    click_mode_toggle
])
affine_input_layout = widgets.VBox([
    widgets.Label("Enter measurement coordinates for the 3 selected points:"),
    widgets.HBox([
        affine_compute_btn,
        widgets.VBox(coord_input_fields),
        sys_input
    ])
])
clear_buttons_layout = widgets.HBox([clear_red_btn, clear_green_btn, save_btn])

widgets_layout = widgets.VBox([
    widgets.HBox([file_uploader, load_image_button]),
    plot_output,
    manual_input_layout, click_input_layout, affine_input_layout,
    clear_buttons_layout,
    report_output,
    status_output
])

display(widgets_layout)

def display_status(msg):
    with status_output:
        clear_output()
        print(msg)

def update_report():
    with report_output:
        clear_output()
        dfs = []
        for df in [calibration_df, measurement_df]:
            if df.empty:
                continue
            expanded = df.copy()
            for system, matrix in affine_matrices:
                transformed = (matrix @ np.vstack([df["original_x"], df["original_y"], np.ones(len(df))])).T
                expanded[f"{system}_x"] = np.round(transformed[:, 0], 1)
                expanded[f"{system}_y"] = np.round(transformed[:, 1], 1)
            dfs.append(expanded)
        if dfs:
            display(pd.concat(dfs, ignore_index=True))

def annotate_point(x, y, label, color='red'):
    point_artist = plt.plot(x, y, '+', color=color)[0]
    label_text = plt.text(
        x + 20, y, label,
        color=color, fontsize=9,
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='none', alpha=0.6)
    )
    if color == 'red':
        red_annotations.append((point_artist, label_text))
    else:
        green_annotations.append((point_artist, label_text))
    fig.canvas.draw()

def compute_affine_matrix(src_pts, dst_pts):
    A, B = [], []
    for (x, y), (u, v) in zip(src_pts, dst_pts):
        A.extend([[x, y, 1, 0, 0, 0], [0, 0, 0, x, y, 1]])
        B.extend([u, v])
    A = np.array(A)
    B = np.array(B)
    params = np.linalg.lstsq(A, B, rcond=None)[0]
    return np.array([[params[0], params[1], params[2]], [params[3], params[4], params[5]]])

def on_canvas_click(event):
    if not event.inaxes or not click_mode_toggle.value:
        return
    x, y = event.xdata, event.ydata
    if len(clicked_points) < 3:
        label = f"CP{len(clicked_points) + 1}"
        clicked_points.append([x, y])
        annotate_point(x, y, label, 'red')
        calibration_df.loc[len(calibration_df)] = [label, x, y]
    else:
        label = f"MP{len(measurement_df) + 1}"
        transformed = {}
        for system, matrix in affine_matrices:
            t = matrix @ np.array([x, y, 1])
            transformed[f"{system}_x"] = t[0]
            transformed[f"{system}_y"] = t[1]
        annotate_point(x, y, label, 'green')
        measurement_df.loc[len(measurement_df)] = {
            "label": label, "original_x": x, "original_y": y, **transformed
        }
    update_report()

def on_manual_add(b):
    if alt_sys_dropdown.value != "None":
        system = alt_sys_dropdown.value
        matrix = next((m for s, m in affine_matrices if s == system), None)
        if matrix is None:
            return
        p = np.array([man_x_input.value, man_y_input.value, 1])
        full_matrix = np.vstack([matrix, [0, 0, 1]])
        x, y, _ = np.linalg.inv(full_matrix) @ p
    else:
        x, y = man_x_input.value, man_y_input.value

    if len(clicked_points) < 3:
        label = f"CP{len(clicked_points) + 1}"
        clicked_points.append([x, y])
        annotate_point(x, y, label, 'red')
        calibration_df.loc[len(calibration_df)] = [label, x, y]
    else:
        label = f"MP{len(measurement_df) + 1}"
        transformed = {}
        for system, matrix in affine_matrices:
            t = matrix @ np.array([x, y, 1])
            transformed[f"{system}_x"] = t[0]
            transformed[f"{system}_y"] = t[1]
        annotate_point(x, y, label, 'green')
        measurement_df.loc[len(measurement_df)] = {
            "label": label, "original_x": x, "original_y": y, **transformed
        }
    update_report()

def on_compute_transform(b):
    if len(clicked_points) != 3:
        return
    real_world_coords = [[box.children[0].value, box.children[1].value] for box in coord_input_fields]
    matrix = compute_affine_matrix(clicked_points, real_world_coords)
    system = sys_input.value.strip()
    if not system:
        return
    affine_matrices.append((system, matrix))
    alt_sys_dropdown.options = ["None"] + [s for s, _ in affine_matrices]

def clear_annotations(annotations):
    for point_artist, label_text in annotations:
        point_artist.remove()
        label_text.remove()
    annotations.clear()
    fig.canvas.draw()

def on_clear_red(b):
    global clicked_points, calibration_df
    clicked_points.clear()
    calibration_df = pd.DataFrame(columns=["label", "original_x", "original_y"])
    clear_annotations(red_annotations)
    update_report()

def on_clear_green(b):
    global measurement_df
    measurement_df = pd.DataFrame(columns=["label", "original_x", "original_y"])
    clear_annotations(green_annotations)
    update_report()

def save_image_and_report(filename="voila_output.png"):
    if fig is None or (calibration_df.empty and measurement_df.empty):
        return
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    save_fig, (ax_img, ax_table) = plt.subplots(1, 2, figsize=(16, 10), gridspec_kw={'width_ratios': [2, 1]})
    ax_img.imshow(img)
    ax_img.set_xlim(xlim)
    ax_img.set_ylim(ylim)
    for point_artist, label_text in red_annotations + green_annotations:
        ax_img.plot(point_artist.get_xdata(), point_artist.get_ydata(), '+', color=point_artist.get_color())
        ax_img.text(label_text.get_position()[0] + 20, label_text.get_position()[1], label_text.get_text(),
                    color=label_text.get_color(), fontsize=9,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='none', alpha=0.6))
    ax_img.axis('off')
    ax_table.axis('off')
    data = pd.concat([calibration_df, measurement_df], ignore_index=True)
    ax_table.table(cellText=data.values, colLabels=data.columns, loc='center')
    save_fig.tight_layout()
    save_fig.savefig(filename)
    plt.close(save_fig)

def on_load_image_clicked(b):
    global fig, ax, img
    if not file_uploader.value:
        display_status("Please upload an image file first.")
        return
    file_content = next(iter(file_uploader.value))['content']
    img = np.array(Image.open(BytesIO(file_content)))
    with plot_output:
        clear_output(wait=True)
        fig = plt.figure(figsize=(8, 8))
        ax = fig.gca()
        plt.imshow(img)
        plt.title("Measurement points", fontsize=14, weight='bold')
        plt.xlabel("X axis", fontsize=12)
        plt.ylabel("Y axis", fontsize=12)
        plt.show()
        fig.canvas.mpl_connect("button_press_event", on_canvas_click)
    display_status("Image loaded. You can now add points.")

# Connect buttons
load_image_button.on_click(on_load_image_clicked)
man_add_btn.on_click(on_manual_add)
affine_compute_btn.on_click(on_compute_transform)
clear_red_btn.on_click(on_clear_red)
clear_green_btn.on_click(on_clear_green)
save_btn.on_click(lambda b: save_image_and_report("voila_output.png"))
